In [10]:
# pip install uv


2.21.3


In [33]:
import mlflow
from mlflow.models import Model
from mlflow.tracking import MlflowClient
from mlflow.models import infer_signature
import conda
import numpy as np

# ===== MLflow tracking =====
TRACKING_URI = "http://sunrise-mlflow-tracking.mlflow.svc.cluster.local:5080"
EXPERIMENT_NAME = "k8s-cpu-forecasting"
REGISTERED_MODEL_NAME = "cpu-pct"     # model registry name (optional but nice)

mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
ml_client = MlflowClient()

print("MLflow:", TRACKING_URI, "Experiment:", EXPERIMENT_NAME)
model_uri = 'runs:/c9eee3c98eee4309a0a602339d1ea612/2025-09-08-11:16:55-cpu-node-1-pct-model'
# The model is logged with an input example
pyfunc_model = mlflow.pyfunc.load_model(model_uri)
input_data = pyfunc_model.input_example


# If you run this from a shell where torch==2.8.0 is already installed:
mlflow.models.predict(
    model_uri=model_uri,
    input_data=input_data,
    env_manager="uv"  # disables virtualenv/conda, uses your CURRENT environment
)


MLflow: http://sunrise-mlflow-tracking.mlflow.svc.cluster.local:5080 Experiment: k8s-cpu-forecasting


2025/09/08 13:29:45 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


2025/09/08 13:29:46 INFO mlflow.utils.virtualenv: Environment /tmp/tmp3qx6g8wx/envs/virtualenv_envs/mlflow-8c3b8977e6a5a31eb2cc96f87c4418cad74670ab already exists
2025/09/08 13:29:46 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /tmp/tmp3qx6g8wx/envs/virtualenv_envs/mlflow-8c3b8977e6a5a31eb2cc96f87c4418cad74670ab/bin/activate && python -c ""']'
2025/09/08 13:29:46 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /tmp/tmp3qx6g8wx/envs/virtualenv_envs/mlflow-8c3b8977e6a5a31eb2cc96f87c4418cad74670ab/bin/activate && python /opt/conda/lib/python3.11/site-packages/mlflow/pyfunc/_mlflow_pyfunc_backend_predict.py --model-uri file:///tmp/tmpx52no5g7/2025-09-08-11%3A16%3A55-cpu-node-1-pct-model --content-type json --input-path /tmp/tmpo4u3wj28/input.json']'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/mlflow/pyfunc/_mlflow_pyfunc_backend_predict.py", line 7, in <module>
    from mlflow.pyfunc.scoring_ser

MlflowException: Non-zero exit code: 1
Command: ['bash', '-c', 'source /tmp/tmp3qx6g8wx/envs/virtualenv_envs/mlflow-8c3b8977e6a5a31eb2cc96f87c4418cad74670ab/bin/activate && python /opt/conda/lib/python3.11/site-packages/mlflow/pyfunc/_mlflow_pyfunc_backend_predict.py --model-uri file:///tmp/tmpx52no5g7/2025-09-08-11%3A16%3A55-cpu-node-1-pct-model --content-type json --input-path /tmp/tmpo4u3wj28/input.json']

An exception occurred while running model prediction within a uv environment. You can find the error message from the prediction subprocess by scrolling above.

In [35]:
## Manual way

import mlflow
import numpy as np
from mlflow.models import Model
from mlflow.tracking import MlflowClient
from mlflow.models import infer_signature
import conda

print(mlflow.__version__)

# ===== MLflow tracking =====
TRACKING_URI = "http://sunrise-mlflow-tracking.mlflow.svc.cluster.local:5080"
EXPERIMENT_NAME = "k8s-cpu-forecasting"
REGISTERED_MODEL_NAME = "cpu-pct"     # model registry name (optional but nice)

mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
ml_client = MlflowClient()

print("MLflow:", TRACKING_URI, "Experiment:", EXPERIMENT_NAME)

model_uri = 'runs:/c9eee3c98eee4309a0a602339d1ea612/2025-09-08-11:16:55-cpu-node-1-pct-model'
model = mlflow.pyfunc.load_model(model_uri)

# Use example input shape from MLflow UI
input_data = np.array([[
    [0.19565217],
    [0.22826087],
    [0.25],
    [0.20652173],
    [0.22826087]
]],dtype=np.float32)   # shape (1, 5, 1)

result = model.predict(input_data)
print(result)

In [32]:
pred = pyfunc_model.predict(input_data)
print(pred)
pred_scalar = float(np.array(pred).squeeze())   # -> just the number
print(pred_scalar)

[[[0.21144302]]]
0.21144302189350128


In [39]:
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
client = MlflowClient()
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)

# Fetch ALL runs, sort by start time DESC (latest first)
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    filter_string="",  # You could filter by tag if you want to
    order_by=["attributes.start_time DESC"],
    max_results=10  # adjust if needed
)

# Find your two target runs by `run_name` or `tags`!
single_model_run = None
hpo_parent_run = None
for run in runs:
    rn = run.data.tags.get("mlflow.runName", "")
    if rn.endswith("cpu-node-1-forecast") and not "hpo" in rn:
        single_model_run = run
    if rn.endswith("hpo-cpu-node-1-forecast-session"):
        hpo_parent_run = run
    # Stop if both found
    if single_model_run and hpo_parent_run:
        break

if not single_model_run or not hpo_parent_run:
    raise Exception("Could not find both single and HPO runs.")

# For the HPO run, get the FINAL nested run (best model)
hpo_child_runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    filter_string=f"tags.mlflow.parentRunId = '{hpo_parent_run.info.run_id}'",
    order_by=["attributes.start_time DESC"],
    max_results=5
)
# Find the nested run whose name contains "final-best" or similar
hpo_best_run = None
for run in hpo_child_runs:
    rn = run.data.tags.get("mlflow.runName", "")
    if "final" in rn or "best" in rn:
        hpo_best_run = run
        break
if not hpo_best_run:
    # fallback: just pick the first child run (most recent)
    hpo_best_run = hpo_child_runs[0]

# Now, get the model URIs
timestamp = single_model_run.data.tags.get("mlflow.runName", "").split("-cpu-node-1-forecast")[0]
single_model_uri = f"runs:/{single_model_run.info.run_id}/{timestamp}-cpu-node-1-pct-model"
hpo_model_uri = f"runs:/{hpo_best_run.info.run_id}/{timestamp}-hpo-cpu-node-1-pct-model"

print("Single model:", single_model_uri)
print("HPO model:", hpo_model_uri)


Single model: runs:/891119da512b479d84ab3fcb6bf4a5fc/2025-09-08-14:25:55-cpu-node-1-pct-model
HPO model: runs:/6311c4290f5f4fd9beb5902a2e3ee304/2025-09-08-14:25:55-hpo-cpu-node-1-pct-model


### dummy stage 4 valid
@dsl.component(
    base_image="docker.io/jhofydu/pytorch-kfp:v1.0.0",
    packages_to_install=["minio", "mlflow==2.21.3", "uv", "virtualenv"]
)
def model_validation() -> NamedTuple("Outputs", [("hpo_cpu_node_1_forecast", str)]):
    '''
    import mlflow
    from mlflow.models import Model
    from mlflow.tracking import MlflowClient
    from mlflow.models import infer_signature
    import conda
    
    # ===== MLflow tracking =====
    TRACKING_URI = "http://sunrise-mlflow-tracking.mlflow.svc.cluster.local:5080"
    EXPERIMENT_NAME = "k8s-cpu-forecasting"
    REGISTERED_MODEL_NAME = "cpu-pct"     # model registry name (optional but nice)
    
    mlflow.set_tracking_uri(TRACKING_URI)
    mlflow.set_experiment(EXPERIMENT_NAME)
    ml_client = MlflowClient()
    
    print("MLflow:", TRACKING_URI, "Experiment:", EXPERIMENT_NAME)
    model_uri = 'runs:/c9eee3c98eee4309a0a602339d1ea612/2025-09-08-11:16:55-cpu-node-1-pct-model'
    # The model is logged with an input example
    pyfunc_model = mlflow.pyfunc.load_model(model_uri)
    input_data = pyfunc_model.input_example
    
    
    # If you run this from a shell where torch==2.8.0 is already installed:
    mlflow.models.predict(
        model_uri=model_uri,
        input_data=input_data,
        env_manager="conda"  # disables virtualenv/conda, uses your CURRENT environment
    )
    '''
    import mlflow
    import numpy as np
    from mlflow.models import Model
    from mlflow.tracking import MlflowClient
    from mlflow.models import infer_signature
    import conda
    
    print(mlflow.__version__)
    
    # ===== MLflow tracking =====
    TRACKING_URI = "http://sunrise-mlflow-tracking.mlflow.svc.cluster.local:5080"
    EXPERIMENT_NAME = "k8s-cpu-forecasting"
    REGISTERED_MODEL_NAME = "cpu-pct"     # model registry name (optional but nice)
    
    mlflow.set_tracking_uri(TRACKING_URI)
    mlflow.set_experiment(EXPERIMENT_NAME)
    ml_client = MlflowClient()
    
    print("MLflow:", TRACKING_URI, "Experiment:", EXPERIMENT_NAME)
    
    model_uri = 'runs:/c9eee3c98eee4309a0a602339d1ea612/2025-09-08-11:16:55-cpu-node-1-pct-model'
    model = mlflow.pyfunc.load_model(model_uri)
    
    # Use example input shape from MLflow UI
    input_data = np.array([[
        [0.19565217],
        [0.22826087],
        [0.25],
        [0.20652173],
        [0.22826087]
    ]],dtype=np.float32)   # shape (1, 5, 1)
    
    result = model.predict(input_data)
    print(result)
    output_str = np.array2string(result)
    return (output_str,)


# 4STAGE implementation

### 1Cell functions

In [48]:
import mlflow
from mlflow.tracking import MlflowClient
import numpy as np

def setup_mlflow(tracking_uri, experiment_name):
    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(experiment_name)
    client = MlflowClient()
    experiment = client.get_experiment_by_name(experiment_name)
    return client, experiment

def find_runs(client, experiment_id, single_suffix="cpu-node-1-forecast", hpo_suffix="hpo-cpu-node-1-forecast-session", limit=10):
    runs = client.search_runs(
        experiment_ids=[experiment_id],
        order_by=["attributes.start_time DESC"],
        max_results=limit
    )
    single_run, hpo_parent_run = None, None
    for run in runs:
        rn = run.data.tags.get("mlflow.runName", "")
        if rn.endswith(single_suffix) and "hpo" not in rn:
            single_run = run
        if rn.endswith(hpo_suffix):
            hpo_parent_run = run
        if single_run and hpo_parent_run:
            break
    if not single_run or not hpo_parent_run:
        raise Exception("Could not find both single and HPO runs.")
    return single_run, hpo_parent_run

def find_final_hpo_child_run(client, experiment_id, parent_run, keyword="final-best"):
    hpo_child_runs = client.search_runs(
        experiment_ids=[experiment_id],
        filter_string=f"tags.mlflow.parentRunId = '{parent_run.info.run_id}'",
        order_by=["attributes.start_time DESC"],
        max_results=10
    )
    for run in hpo_child_runs:
        rn = run.data.tags.get("mlflow.runName", "")
        if keyword in rn or "final" in rn or "best" in rn:
            return run
    return hpo_child_runs[0] if hpo_child_runs else None

def build_model_uris(single_run, hpo_run, suffix="-cpu-node-1-pct-model", hpo_suffix="-hpo-cpu-node-1-pct-model"):
    timestamp = single_run.data.tags.get("mlflow.runName", "").split("-cpu-node-1-forecast")[0]
    single_model_uri = f"runs:/{single_run.info.run_id}/{timestamp}{suffix}"
    hpo_model_uri = f"runs:/{hpo_run.info.run_id}/{timestamp}{hpo_suffix}"
    print("Single model uri:", single_model_uri)
    print("HPO model uri:", hpo_model_uri)
    return single_model_uri, hpo_model_uri

def validate_model(model_uri, example_input):
    import mlflow
    model = mlflow.pyfunc.load_model(model_uri)
    result = model.predict(example_input)
    print(f"Result for {model_uri}: {result}")
    return result




### 2 Cell Calling 

In [51]:

TRACKING_URI = "http://sunrise-mlflow-tracking.mlflow.svc.cluster.local:5080"
EXPERIMENT_NAME = "k8s-cpu-forecasting"

client, experiment = setup_mlflow(TRACKING_URI, EXPERIMENT_NAME)
single_run, hpo_parent_run = find_runs(client, experiment.experiment_id)
hpo_best_run = find_final_hpo_child_run(client, experiment.experiment_id, hpo_parent_run)

single_model_uri, hpo_model_uri = build_model_uris(single_run, hpo_best_run)

# Example input (replace with your real validation input!)
input_data = np.array([[
    [0.19565217],
    [0.22826087],
    [0.25],
    [0.20652173],
    [0.22826087]
]], dtype=np.float32)

single_result = validate_model(single_model_uri, input_data)
hpo_result = validate_model(hpo_model_uri, input_data)
single_result_scalar = float(np.array(single_result).squeeze())   # -> just the number
hpo_result_scalar = float(np.array(hpo_result).squeeze())   # -> just the number
print("Single model output prediction:", single_result_scalar)
print("HPO model output:prediction", hpo_result_scalar)



Single model uri: runs:/ed180661112f422ea9b9513ecdf84a94/2025-09-08-14:36:04-cpu-node-1-pct-model
HPO model uri: runs:/cb82d7a3937147e98a844b2e649705db/2025-09-08-14:36:04-hpo-cpu-node-1-pct-model


Result for runs:/ed180661112f422ea9b9513ecdf84a94/2025-09-08-14:36:04-cpu-node-1-pct-model: [[[0.22926752]]]


Result for runs:/cb82d7a3937147e98a844b2e649705db/2025-09-08-14:36:04-hpo-cpu-node-1-pct-model: [[[0.21611929]]]
Single model output prediction: 0.22926752269268036
HPO model output:prediction 0.21611928939819336
